In [1]:
#Importing libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
from numpy import asarray
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
from statsmodels.tsa.tsatools import lagmat
from statsmodels.tsa.arima.model import ARIMA

# Data Import and Cleaning

In [2]:
df = pd.read_excel("/Users/surajkwork/Documents/Thesis/thesis/df.xlsx")

The date variable is not in datetime format so it has to be converted.

In [3]:
df['date']= pd.to_datetime(df['date'])
df.index = df['date']
df.sort_index(inplace=True)

Extracting one cluster from the dataset for analysis.

In [4]:
TS1 = df[(df['Plz'] == "['25', '24']")]

And from this cluster, extracting one category for analysis.

In [5]:
TS1_A2 = TS1[TS1['full'] == 'A2 -  geschreddert']

In [6]:
TS1_A1A2 = TS1[TS1['full'] == 'A1 & A2 -  geschreddert']
TS1_A2A3 = TS1[TS1['full'] == 'A2 & A3 -  geschreddert']

In [7]:
datasets = [TS1_A2, TS1_A1A2, TS1_A2A3]

# Modelling - Train, test split

In [8]:
X = TS1_A1A2['wPreis'].values

In [9]:
train, test = X[0:-6], X[-6:]

In [10]:
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=4):
	df = pd.DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = pd.concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df

In [11]:
supervised = timeseries_to_supervised(X, 4)
print(supervised.head())

           0          0          0          0          0
0   0.000000   0.000000   0.000000   0.000000 -37.519318
1 -37.519318   0.000000   0.000000   0.000000 -28.690096
2 -28.690096 -37.519318   0.000000   0.000000 -29.688079
3 -29.688079 -28.690096 -37.519318   0.000000 -25.630433
4 -25.630433 -29.688079 -28.690096 -37.519318 -29.073794


In [12]:
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return pd.Series(diff)

In [13]:
# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

In [14]:
# transform to be stationary
series = X
differenced = difference(series, 1)
print(differenced.head())
# invert transform
inverted = list()
for i in range(len(differenced)):
 value = inverse_difference(series, differenced[i], len(series)-i)
 inverted.append(value)
inverted = pd.Series(inverted)
print(inverted.head())

0    8.829222
1   -0.997983
2    4.057645
3   -3.443361
4    4.137368
dtype: float64
0   -28.690096
1   -29.688079
2   -25.630433
3   -29.073794
4   -24.936426
dtype: float64


In [15]:
differenced.shape

(153,)

In [16]:
inverted.shape

(153,)

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
X_2 = X.reshape(len(X), 1)
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(X_2)
scaled_X = scaler.transform(X_2)

In [19]:
scaled_series = pd.Series(scaled_X[:, 0])
print(scaled_series.head())
# invert transform
inverted_X = scaler.inverse_transform(scaled_X)
inverted_series = pd.Series(inverted_X[:, 0])
print(inverted_series.head())

0   -1.000000
1   -0.826590
2   -0.846191
3   -0.766496
4   -0.834126
dtype: float64
0   -37.519318
1   -28.690096
2   -29.688079
3   -25.630433
4   -29.073794
dtype: float64


In [20]:
def scale(train, test):
 # fit scaler
 scaler = MinMaxScaler(feature_range=(-1, 1))
 scaler = scaler.fit(train)
 # transform train
 train = train.reshape(train.shape[0], train.shape[1])
 train_scaled = scaler.transform(train)
 # transform test
 test = test.reshape(test.shape[0], test.shape[1])
 test_scaled = scaler.transform(test)
 return scaler, train_scaled, test_scaled

In [21]:
# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
 new_row = [x for x in X] + [value]
 array = np.array(new_row)
 array = array.reshape(1, len(array))
 inverted = scaler.inverse_transform(array)
 return inverted[0, -1]

## LSTM

In [1]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [18]:
import tensorflow.keras as keras

ModuleNotFoundError: No module named 'tensorflow.keras'

In [12]:
import keras

ModuleNotFoundError: No module named 'tensorflow.compat'

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'tensorflow.keras'